### **Setting GEE environnement**
Due to compatibility issues it is recommended to use a dedicated environment for the geemap library.

See instructions on how to do this here: https://geemap.org/installation/

In [2]:
# Install the libraries if they are not installed (you can remove these lines if you have already installed them)
# !pip install earthengine-api geemap

# Charging the libraries
import ee
import geemap

# Authenticate and initialize Earth Engine session
# ee.Authenticate()  # Only required the first time or when the access token expires
ee.Initialize()

### **Visualisation mode for one roi in one year_of_interest**

In [22]:
# Define the ROI and load the dataset
roi = ee.FeatureCollection('users/caslumali/UE801_Stage/Cotriguacu') # change the name of the ROIi : 'Cotriguacu' / 'Guaviare' / 'MDD' / 'Paragominas'
terraClimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")

In [29]:
def calculate_annual_temp_avg(year, roi):
    """Calculate the annual temperatura average for a specific climate year."""
    startDate = ee.Date.fromYMD(year, 8, 1)
    endDate = ee.Date.fromYMD(year + 1, 7, 31)
    filteredYear = terraClimate.filterDate(startDate, endDate).map(
        lambda image: image.select(['tmmx', 'tmmn']).multiply(0.1).reduce(ee.Reducer.mean()).clip(roi)
    )
    annual_temp_avg = filteredYear.reduce(ee.Reducer.mean()).rename('Temperature_avg')
    return annual_temp_avg

def calculate_stats(image, roi):
    """Calculate max, min, and average temperature within the ROI."""
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax().combine(reducer2=ee.Reducer.mean(), sharedInputs=True),
        geometry=roi.geometry(),
        scale=4638,
        maxPixels=1e9
    )
    return stats.getInfo()

# Year of interest for the temperature analysis
year_of_interest = 2022 # change the date for stats and visualisation

# Calculate the annual cumulative precipitation
annual_temp_avg = calculate_annual_temp_avg(year_of_interest, roi)

# Calculate and print temperature statistics
stats = calculate_stats(annual_temp_avg, roi)
print(f"Temperature values for {year_of_interest} - {year_of_interest + 1}\n", stats)


Temperature values for 2022 - 2023
 {'Temperature_avg_max': 27.2, 'Temperature_avg_mean': 26.56315086851269, 'Temperature_avg_min': 25.954166666666662}


In [30]:
# Calculate CHIRPS statistics for the year of interest to generate dynamic visualization
stats = calculate_stats(annual_temp_avg, roi)

# Define visualization parameters based on the calculated statistics
precipitation_vis = {
    'min': stats['Temperature_avg_min'],
    'max': stats['Temperature_avg_max'],
    'palette': ['1a3678', '2955bc', '5699ff', '8dbae9', 'acd1ff', 'caebff', 'e5f9ff',
    'fdffb4', 'ffe6a2', 'ffc969', 'ffa12d', 'ff7c1f', 'ca531a', 'ff0000',
    'ab0000'
  ],
}

# View the Temperature average on the map
Map = geemap.Map()
Map.addLayer(annual_temp_avg, precipitation_vis, 'Annual Temperature Average')
Map.centerObject(roi, 7) # Change the zoom depending on the area
Map

Map(center=[-3.184380688531968, -47.60048105896091], controls=(WidgetControl(options=['position', 'transparent…

### **Looping for export average temperature for each ROI**

In [31]:
# Define ROIs in a list
# Export ROIs have a 10km buffer
rois = {
    'Cotriguacu': ee.FeatureCollection('users/caslumali/UE801_Stage/Cotriguacu_10km'),
    'Guaviare': ee.FeatureCollection('users/caslumali/UE801_Stage/Guaviare_10km'),
    'MDD': ee.FeatureCollection('users/caslumali/UE801_Stage/MDD_10km'),
    'Paragominas': ee.FeatureCollection('users/caslumali/UE801_Stage/Paragominas_10km'),
}

In [32]:
from datetime import datetime

# Get current year
current_year = datetime.now().year

# Loop for each ROI
for roi_name, roi in rois.items():
    print(f'Processing {roi_name}...')
    for year in range(2000, current_year):  # Goes to the current year
        # If we are in the current year and processing the last year, skip the combination
        if year == current_year - 1:
            continue  # Skip to the next iteration of the loop

        # Calculate annual temperature average
        annual_temp_avg = calculate_annual_temp_avg(year, roi).toDouble()
        
        # Set up export parameters.
        description = f'{roi_name}_Annual_TempAvg_{year}_{year + 1}'
        file_name_prefix = f'{roi_name}_Annual_TempAvg_{year}_{year + 1}'

        # Start export task
        task = ee.batch.Export.image.toDrive(
            image=annual_temp_avg,
            description=description,
            fileNamePrefix=file_name_prefix,
            folder=f'TempAvg_{roi_name}',
            scale=4638,
            region=roi.geometry(),
            crs='EPSG:4326',
            fileFormat='GeoTIFF',
            maxPixels=1e9
        )
        task.start()
        print(f'Export task started for {roi_name} in the year: {year}')

Processing Cotriguacu...
Export task started for Cotriguacu in the year: 2000
Export task started for Cotriguacu in the year: 2001
Export task started for Cotriguacu in the year: 2002
Export task started for Cotriguacu in the year: 2003
Export task started for Cotriguacu in the year: 2004
Export task started for Cotriguacu in the year: 2005
Export task started for Cotriguacu in the year: 2006
Export task started for Cotriguacu in the year: 2007
Export task started for Cotriguacu in the year: 2008
Export task started for Cotriguacu in the year: 2009
Export task started for Cotriguacu in the year: 2010
Export task started for Cotriguacu in the year: 2011
Export task started for Cotriguacu in the year: 2012
Export task started for Cotriguacu in the year: 2013
Export task started for Cotriguacu in the year: 2014
Export task started for Cotriguacu in the year: 2015
Export task started for Cotriguacu in the year: 2016
Export task started for Cotriguacu in the year: 2017
Export task started f